# Install pyspark library

In [100]:
!pip install pyspark

In [101]:
#import pyspark library
import pyspark

In [102]:
#import spark session library
from pyspark.sql import SparkSession

In [103]:
# Create SparkSession object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('multi_regression') \
                    .getOrCreate()

# Create DataFrame

In [104]:
#To create dataframe form External datasets
df = spark.read.option("header", "true").csv("/Users/krishnapratap/Desktop/partation/data/*")

In [105]:
df.show()

+----+-------+-----+----------+---------+----------+-------------+---------+-------+-------+---------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+---------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----------------+-------------+-----------+-----------+-------------+--------------

22/04/26 14:43:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, UniqueCarrier, AirlineID, Carrier, TailNum, FlightNum, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSDepTime, DepTime, DepDelay, DepDelayMinutes, DepDel15, DepartureDelayGroups, DepTimeBlk, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, ArrDelay, ArrDelayMinutes, ArrDel15, ArrivalDelayGroups, ArrTimeBlk, Cancelled, CancellationCode, Diverted, CRSElapsedTime, ActualElapsedTime, AirTime, Flights, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, FirstDepTime, TotalAddGTime, LongestAddGTime, DivAirportLandings, DivReachedDest, DivActualElapsedTime, DivArrDelay, DivDistan

# Data Preparation

In [106]:
#create new dataframe as per required columns for prediction
AirlineDF = df.select("Origin","Dest","AirTime","Distance","DepDelayMinutes","ArrDelayMinutes")

In [107]:
#cache data in-memory
AirlineDF.cache()

DataFrame[Origin: string, Dest: string, AirTime: string, Distance: string, DepDelayMinutes: string, ArrDelayMinutes: string]

In [108]:
AirlineDF.show()

+------+----+-------+--------+---------------+---------------+
|Origin|Dest|AirTime|Distance|DepDelayMinutes|ArrDelayMinutes|
+------+----+-------+--------+---------------+---------------+
|   JFK| LAX| 338.00| 2475.00|           1.00|           4.00|
|   JFK| LAX| 349.00| 2475.00|           0.00|          26.00|
|   JFK| LAX| 370.00| 2475.00|           1.00|          28.00|
|   JFK| LAX| 350.00| 2475.00|           0.00|          56.00|
|   JFK| LAX| 335.00| 2475.00|           0.00|           0.00|
|   JFK| LAX| 336.00| 2475.00|           0.00|           0.00|
|   JFK| LAX| 380.00| 2475.00|           0.00|          35.00|
|   JFK| LAX| 359.00| 2475.00|           0.00|          17.00|
|   JFK| LAX| 368.00| 2475.00|           0.00|          58.00|
|   JFK| LAX| 356.00| 2475.00|           0.00|           7.00|
|   JFK| LAX| 353.00| 2475.00|           0.00|           7.00|
|   JFK| LAX| 332.00| 2475.00|           0.00|           0.00|
|   JFK| LAX| 339.00| 2475.00|           0.00|         

In [109]:
#check data types of each columns
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- DepDelayMinutes: string (nullable = true)
 |-- ArrDelayMinutes: string (nullable = true)



# We need to change data types of columns

In [110]:
#import sql library for data types 
from pyspark.sql.types import IntegerType ,DoubleType

In [111]:
AirlineDF = AirlineDF.withColumn("Distance", AirlineDF["Distance"].cast(IntegerType()))
AirlineDF = AirlineDF.withColumn("AirTime", AirlineDF["AirTime"].cast(IntegerType()))
AirlineDF = AirlineDF.withColumn("DepDelayMinutes", AirlineDF["DepDelayMinutes"].cast(DoubleType()))
AirlineDF = AirlineDF.withColumn("ArrDelayMinutes", AirlineDF["ArrDelayMinutes"].cast(DoubleType()))

In [112]:
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)



# Check is there any null values in DataFrame 

In [113]:
#import libray for sql function col
from pyspark.sql.functions import col

In [114]:
#check null values in each columns
print(AirlineDF.where(col("Origin").isNull()).count())
print(AirlineDF.where(col("Dest").isNull()).count())
print(AirlineDF.where(col("AirTime").isNull()).count())
print(AirlineDF.where(col("Distance").isNull()).count())
print(AirlineDF.where(col("DepDelayMinutes").isNull()).count())
print(AirlineDF.where(col("ArrDelayMinutes").isNull()).count())

0
0
183789


0


166818


183789


# we need to handel null values 

In [115]:
#now drop row as correspondence to null values
AirlineDF = AirlineDF.dropna()

In [116]:
#check null values in each columns
print(AirlineDF.where(col("Origin").isNull()).count())
print(AirlineDF.where(col("Dest").isNull()).count())
print(AirlineDF.where(col("AirTime").isNull()).count())
print(AirlineDF.where(col("Distance").isNull()).count())
print(AirlineDF.where(col("DepDelayMinutes").isNull()).count())
print(AirlineDF.where(col("ArrDelayMinutes").isNull()).count())

0
0


0


0


0


0


In [117]:
AirlineDF.show()

+------+----+-------+--------+---------------+---------------+
|Origin|Dest|AirTime|Distance|DepDelayMinutes|ArrDelayMinutes|
+------+----+-------+--------+---------------+---------------+
|   JFK| LAX|    338|    2475|            1.0|            4.0|
|   JFK| LAX|    349|    2475|            0.0|           26.0|
|   JFK| LAX|    370|    2475|            1.0|           28.0|
|   JFK| LAX|    350|    2475|            0.0|           56.0|
|   JFK| LAX|    335|    2475|            0.0|            0.0|
|   JFK| LAX|    336|    2475|            0.0|            0.0|
|   JFK| LAX|    380|    2475|            0.0|           35.0|
|   JFK| LAX|    359|    2475|            0.0|           17.0|
|   JFK| LAX|    368|    2475|            0.0|           58.0|
|   JFK| LAX|    356|    2475|            0.0|            7.0|
|   JFK| LAX|    353|    2475|            0.0|            7.0|
|   JFK| LAX|    332|    2475|            0.0|            0.0|
|   JFK| LAX|    339|    2475|            0.0|         

# we need convert distance miles into kilo meter

In [118]:
from pyspark.sql.functions import round

# Convert 'mile' to 'km' 
AirlineDF = AirlineDF.withColumn('Distance', round(AirlineDF.Distance * 1.60934, 0))

AirlineDF.show()

+------+----+-------+--------+---------------+---------------+
|Origin|Dest|AirTime|Distance|DepDelayMinutes|ArrDelayMinutes|
+------+----+-------+--------+---------------+---------------+
|   JFK| LAX|    338|  3983.0|            1.0|            4.0|
|   JFK| LAX|    349|  3983.0|            0.0|           26.0|
|   JFK| LAX|    370|  3983.0|            1.0|           28.0|
|   JFK| LAX|    350|  3983.0|            0.0|           56.0|
|   JFK| LAX|    335|  3983.0|            0.0|            0.0|
|   JFK| LAX|    336|  3983.0|            0.0|            0.0|
|   JFK| LAX|    380|  3983.0|            0.0|           35.0|
|   JFK| LAX|    359|  3983.0|            0.0|           17.0|
|   JFK| LAX|    368|  3983.0|            0.0|           58.0|
|   JFK| LAX|    356|  3983.0|            0.0|            7.0|
|   JFK| LAX|    353|  3983.0|            0.0|            7.0|
|   JFK| LAX|    332|  3983.0|            0.0|            0.0|
|   JFK| LAX|    339|  3983.0|            0.0|         

In [119]:
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- Distance: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)



## Verctorize the features

In [120]:
from pyspark.ml.feature import *

In [121]:
from pyspark.ml.feature import VectorAssembler

In [122]:
vectorizer = VectorAssembler()
vectorizer.setInputCols(['Distance','DepDelayMinutes','ArrDelayMinutes'])
vectorizer.setOutputCol('features')

df_vect = vectorizer.setHandleInvalid("keep").transform(AirlineDF)

In [123]:
df_vect.show()

+------+----+-------+--------+---------------+---------------+-----------------+
|Origin|Dest|AirTime|Distance|DepDelayMinutes|ArrDelayMinutes|         features|
+------+----+-------+--------+---------------+---------------+-----------------+
|   JFK| LAX|    338|  3983.0|            1.0|            4.0| [3983.0,1.0,4.0]|
|   JFK| LAX|    349|  3983.0|            0.0|           26.0|[3983.0,0.0,26.0]|
|   JFK| LAX|    370|  3983.0|            1.0|           28.0|[3983.0,1.0,28.0]|
|   JFK| LAX|    350|  3983.0|            0.0|           56.0|[3983.0,0.0,56.0]|
|   JFK| LAX|    335|  3983.0|            0.0|            0.0| [3983.0,0.0,0.0]|
|   JFK| LAX|    336|  3983.0|            0.0|            0.0| [3983.0,0.0,0.0]|
|   JFK| LAX|    380|  3983.0|            0.0|           35.0|[3983.0,0.0,35.0]|
|   JFK| LAX|    359|  3983.0|            0.0|           17.0|[3983.0,0.0,17.0]|
|   JFK| LAX|    368|  3983.0|            0.0|           58.0|[3983.0,0.0,58.0]|
|   JFK| LAX|    356|  3983.

In [124]:
print(vectorizer.explainParams())

handleInvalid: How to handle invalid data (NULL and NaN values). Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), or 'keep' (return relevant number of NaN in the output). Column lengths are taken from the size of ML Attribute Group, which can be set using `VectorSizeHint` in a pipeline before `VectorAssembler`. Column lengths can also be inferred from first rows of the data since it is safe to do so but only in case of 'error' or 'skip'). (default: error, current: keep)
inputCols: input column names. (current: ['Distance', 'DepDelayMinutes', 'ArrDelayMinutes'])
outputCol: output column name. (default: VectorAssembler_0dd0a41653fd__output, current: features)


# Train Test data Splitting

In [125]:
flights_train, flights_test = df_vect.randomSplit([0.8, 0.2])

In [126]:
flights_train.show()

+------+----+-------+--------+---------------+---------------+-----------------+
|Origin|Dest|AirTime|Distance|DepDelayMinutes|ArrDelayMinutes|         features|
+------+----+-------+--------+---------------+---------------+-----------------+
|   ABE| CLT|     72|   774.0|           19.0|            4.0| [774.0,19.0,4.0]|
|   ABE| CLT|     74|   774.0|            0.0|            0.0|  [774.0,0.0,0.0]|
|   ABE| CLT|     76|   774.0|            0.0|            0.0|  [774.0,0.0,0.0]|
|   ABE| CLT|     77|   774.0|            0.0|            0.0|  [774.0,0.0,0.0]|
|   ABE| CLT|     77|   774.0|            0.0|            6.0|  [774.0,0.0,6.0]|
|   ABE| CLT|     77|   774.0|           50.0|           31.0|[774.0,50.0,31.0]|
|   ABE| CLT|     78|   774.0|            5.0|            2.0|  [774.0,5.0,2.0]|
|   ABE| CLT|     79|   774.0|            0.0|            0.0|  [774.0,0.0,0.0]|
|   ABE| CLT|     79|   774.0|            0.0|            0.0|  [774.0,0.0,0.0]|
|   ABE| CLT|     82|   774.

## Regression Model Training

In [127]:
from pyspark.ml.regression import LinearRegression

In [128]:
lr = LinearRegression()
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0. (default: 0.0)
maxIter: max number of iterations (>= 0). (default: 100)
predic

In [129]:
lr.setLabelCol("AirTime")
lr.setFeaturesCol("features")
model = lr.fit(flights_train)

22/04/26 14:44:20 WARN Instrumentation: [f105e186] regParam is zero, which might cause numerical instability and overfitting.


In [130]:
type(model)

pyspark.ml.regression.LinearRegressionModel

# View model summary

In [131]:
print("R2:", model.summary.r2)
print("Intercept: ", model.intercept, "Coefficients", model.coefficients)

R2: 0.9615997031185551
Intercept:  17.542500619849186 Coefficients [0.07339754515234137,-0.3721378545050592,0.38038335821374736]


# Model Testing 

In [132]:
df_pred = model.transform(flights_test)
df_pred.show()

+------+----+-------+--------+---------------+---------------+-----------------+------------------+
|Origin|Dest|AirTime|Distance|DepDelayMinutes|ArrDelayMinutes|         features|        prediction|
+------+----+-------+--------+---------------+---------------+-----------------+------------------+
|   ABE| CLT|     70|   774.0|            3.0|           10.0| [774.0,3.0,10.0]|  77.0396205863837|
|   ABE| CLT|     77|   774.0|            0.0|            0.0|  [774.0,0.0,0.0]|  74.3522005677614|
|   ABE| CLT|     82|   774.0|            0.0|            0.0|  [774.0,0.0,0.0]|  74.3522005677614|
|   ABE| CLT|     84|   774.0|            4.0|            7.0|  [774.0,4.0,7.0]|  75.5263326572374|
|   ABE| CLT|     86|   774.0|            0.0|            0.0|  [774.0,0.0,0.0]|  74.3522005677614|
|   ABE| CLT|     87|   774.0|            0.0|            0.0|  [774.0,0.0,0.0]|  74.3522005677614|
|   ABE| CLT|     87|   774.0|            0.0|           11.0| [774.0,0.0,11.0]| 78.53641750811263|
